In [1]:
import pandas as pd
import numpy as np
import json
import mysql.connector
import math
import csv
import os
import pytz

In [2]:
imp_files = []
file_path = r'D:\\sriram\\agrud\\cbonds_data_entry\\test_files\\historical_data\\historical_data\\quotes_archive\\'
files = os.listdir(file_path)
start_pt = 2200
end_pt = 2250

In [3]:
with open('isin_to_masterid_map.json', 'r') as f:
    isinToMasterid = json.load(f)
with open('col_to_indicator_map.json', 'r', encoding='utf-8') as f:
    colToIndicator = json.load(f)

In [4]:
stock_exchange_priority = ('Frankfurt S.E.','Stuttgart Exchange','Berlin Exchange','Dusseldorf SE','FINRA TRACE','Cbonds Estimation','Luxembourg S.E','MiFID II Source 2 (APA, Post-trade reporting)','MiFID II Source 1 (APA, Post-trade reporting)','Hong Kong S.E.','SGX','US OTC Market','Other sources of prices','London S.E.','Euronext Paris','Taipei Exchange (OTC)','Nasdaq Dubai','Taipei Exchange (Trading System)','FedInvest')
close_price_priority = ['close price', 'indicative price', 'bid (at close)', 'ask (at close)']
dateColumns = ('trade date','put/сall date',"put/\u0441all date",'maturity date')

In [5]:
for file in files[start_pt:end_pt]:
    print(file.strip('.xls'))

XS1076957700
XS1077882121
XS1077883012
XS1078662563
XS1078750004
XS1079028566
XS1079527211
XS1079726334
XS1079848369
XS1081321595
XS1082471423
XS1082971588
XS1083844503
XS1083986718
XS1084368593
XS1084942470
XS1086900898
XS1088515207
XS1090334050
XS1090864528
XS1090889947
XS1094768469
XS1097953050
XS1104029290
XS1105268228
XS1106137687
XS1106513762
XS1107291541
XS1108784510
XS1111123987
XS1113141441
XS1115459528
XS1115498260
XS1115502988
XS1115524016
XS1117293107
XS1117297355
XS1120081283
XS1120403313
XS1120608713
XS1120937617
XS1121908211
XS1125272143
XS1128996425
XS1130058743
XS1130913558
XS1134541306
XS1134541561
XS1138283152
XS1138283236


In [6]:
# publish_date = "2022-01-11"
# publish_date = (datetime.datetime.strptime(publish_date, '%Y-%m-%d').date() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")

df = pd.DataFrame()
not_insert = []
with open("not_insert_isin.csv","r") as file:
    csvreader = csv.reader(file)
    for row in csvreader:
        not_insert.append(row[0])
# count = 1
for file in files[start_pt:end_pt]:
# for file in files:
    if file.strip('.xls') in not_insert:
        continue
    # print(file)
    try:
        df2 = pd.read_excel(file_path+file+'.xls')
    except FileNotFoundError:
        df2 = pd.read_excel(file_path+file)
    except Exception as e:
        print(e)
    # print(count)
    # count+=1
    df2.columns = df2.columns.str.strip().str.lower()
    df2['ts date'] = df2['trade date']
    df2['master id'] = df2['isin / isin regs'].map(isinToMasterid)
    df2 = df2[df2['stock exchange (eng)'].isin(stock_exchange_priority)]
    df = df.append(df2)

In [7]:
df['master id'] = df['master id'].astype('int')
df['custom close price col'] = np.nan
for k in stock_exchange_priority:
    for col in close_price_priority:
        df.loc[(df['stock exchange (eng)']==k) & (df['custom close price col'].isna()) 
        & (df[col].notna()), 'custom close price col'] = df.loc[(df['stock exchange (eng)']==k) & (df['custom close price col'].isna()) & (df[col].notna()), col]

In [8]:
preprocessedData = pd.DataFrame()
issueData = []
i = 0
for by, groupedDf in df.groupby(by = ['master id','ts date']):
    # print(i)
    master_id = int(by[0])
    ts_date = by[1]
    for stock_exchange in stock_exchange_priority:
        close_price_col = 'custom close price col'
        stock_exchange_wise_df = groupedDf[groupedDf['stock exchange (eng)'] == stock_exchange]
        if (len(stock_exchange_wise_df) > 0) and (stock_exchange_wise_df[close_price_col].notna().sum()):
            preprocessedData = preprocessedData.append(stock_exchange_wise_df)
            break
    else:
        issueData.append([str(ts_date)])
    i+=1

In [9]:
missing_dates = []
for x in issueData:
    missing_dates.append(x[0])
miss_dict = {}
miss_dict[file] = missing_dates
file1 = open("isin_missing_dates.json",mode="a",encoding="utf-8")
file1.write(json.dumps(miss_dict)+'\n')
file1.close()

In [10]:
close_price_col = 'custom close price col'
preprocessedData.loc[preprocessedData['open price'].isna(), 'open price'] = preprocessedData.loc[preprocessedData['open price'].isna(), close_price_col]
preprocessedData.loc[preprocessedData['maximum price'].isna(), 'maximum price'] = preprocessedData.loc[preprocessedData['maximum price'].isna(), close_price_col]
preprocessedData.loc[preprocessedData['minimum price'].isna(), 'minimum price'] = preprocessedData.loc[preprocessedData['minimum price'].isna(), close_price_col]
preprocessedData.loc[preprocessedData['close price'].isna(), 'close price'] = preprocessedData.loc[preprocessedData['close price'].isna(), close_price_col]

In [11]:
preprocessedData = preprocessedData.replace([np.NaN], ['NA'])
preprocessedData = preprocessedData.replace([pd.NaT], ['NA'])

In [12]:
result = []
for i, row in preprocessedData.iterrows():
    row2 = row.to_dict()
    masterId = row2['master id']
    ts_date = row2['ts date'].strftime("%Y-%m-%d")
    for k, v in row2.items():
        if v == "NA":
            continue
        if k in colToIndicator:
            try:
                indicatorId = colToIndicator[k]
                if type(v) == pd.Timestamp:
                    json_data = None
                    dataType = 2
                    gmt = pytz.timezone("GMT")
                    value_data = gmt.localize(v).timestamp()
                elif type(v) == float or type(v) == int or  v.isnumeric(): 
                    dataType = 0
                    value_data = v
                    json_data = None
                elif v.isnumeric() == False:
                    json_data = json.dumps({'TEXT':v})
                    dataType = 3
                    value_data = 0
            except Exception as e:
                print(k)
                print(v)
                print(e)
            result.append([masterId,indicatorId,value_data,json_data,dataType, ts_date])

In [13]:
len(result)

1666687

In [14]:
# insert query
# try:
#     db_conn = mysql.connector.connect(host='54.237.79.6',user='rentech_user',database = 'rentech_db',password='N)baegbgqeiheqfi3e9314jnEkekjb',auth_plugin='mysql_native_password')
#     cursor = db_conn.cursor()
#     sql = """INSERT INTO `raw_data` (`id`, `master_id`, `indicator_id`, `value_data`, `json_data`, `data_type`, `ts_date`, `ts_hour`, `job_id`, `timestamp`) VALUES 
#     (NULL, %s, %s, %s, %s, %s, %s, '0:0:0', 9, NOW()) ON DUPLICATE KEY UPDATE  
#     master_id = VALUES(master_id), indicator_id = VALUES(indicator_id), value_data = VALUES(value_data), json_data = VALUES(json_data),data_type = VALUES(data_type), ts_date = VALUES(ts_date) ,ts_hour = VALUES(ts_hour),
#     job_id = VALUES(job_id), batch_id = VALUES(batch_id);"""
#     cursor.executemany(sql, result)
#     print (f"{cursor.rowcount} rows are inserted into db")
#     db_conn.commit()
# except Exception as e:
#     print ("Mysql Error:",e)
# finally:
#     if(db_conn.is_connected()):
#             cursor.close()
#             db_conn.close()
#             print("MySQL connection is closed")

In [15]:
for i in range(1, math.ceil(len(result)/200000) + 1):
    start = (i-1) * 200000
    end = i * 200000
    print(start, end)
    # insert query
    try:
        db_conn = mysql.connector.connect(host='54.237.79.6',user='rentech_user',database = 'rentech_db',password='N)baegbgqeiheqfi3e9314jnEkekjb',auth_plugin='mysql_native_password')
        cursor = db_conn.cursor()
        sql = """INSERT INTO `raw_data` (`id`, `master_id`, `indicator_id`, `value_data`, `json_data`, `data_type`, `ts_date`, `ts_hour`, `job_id`, `timestamp`) VALUES 
        (NULL, %s, %s, %s, %s, %s, %s, '0:0:0', 9, NOW()) ON DUPLICATE KEY UPDATE  
        master_id = VALUES(master_id), indicator_id = VALUES(indicator_id), value_data = VALUES(value_data), json_data = VALUES(json_data),data_type = VALUES(data_type), ts_date = VALUES(ts_date) ,ts_hour = VALUES(ts_hour),
        job_id = VALUES(job_id), batch_id = VALUES(batch_id);"""
        cursor.executemany(sql, result[start:end])
        print (f"{cursor.rowcount} rows are inserted into db")
        db_conn.commit()
    except Exception as e:
        print ("Mysql Error:",e,"-")
    finally:
        if(db_conn.is_connected()):
            cursor.close()
            db_conn.close()
            print("MySQL connection is closed")

0 200000
200000 rows are inserted into db
MySQL connection is closed
200000 400000
200000 rows are inserted into db
MySQL connection is closed
400000 600000
200000 rows are inserted into db
MySQL connection is closed
600000 800000
200000 rows are inserted into db
MySQL connection is closed
800000 1000000
200000 rows are inserted into db
MySQL connection is closed
1000000 1200000
200000 rows are inserted into db
MySQL connection is closed
1200000 1400000
200000 rows are inserted into db
MySQL connection is closed
1400000 1600000
200000 rows are inserted into db
MySQL connection is closed
1600000 1800000
66687 rows are inserted into db
MySQL connection is closed
